## Resource Catalogue demo

In [ ]:
from owslib.csw import CatalogueServiceWeb
from geolinks import sniff_link
from owslib.fes import And, Or, PropertyIsEqualTo, PropertyIsGreaterThanOrEqualTo, PropertyIsLessThanOrEqualTo, PropertyIsLike, BBox, SortBy, SortProperty
import folium

### System Catalogue

In [ ]:
system_endpoint = 'https://resource-catalogue.185.52.193.87.nip.io/'

In [ ]:
csw = CatalogueServiceWeb(system_endpoint,timeout=30)

In [ ]:
csw.identification.type

In [ ]:
csw.version

In [ ]:
[op.name for op in csw.operations]

In [ ]:
csw.getdomain('GetRecords.resultType')
csw.results

In [ ]:
csw.get_operation_by_name('GetRecords').constraints

In [ ]:
csw.getrecords2(maxrecords=20)
csw.results

In [ ]:
for rec in csw.records:
    print(f'identifier: {csw.records[rec].identifier}\ntype: {csw.records[rec].type}\ntitle: {csw.records[rec].title}\n')

In [ ]:
bbox_query = BBox([37.8, 23.4, 38.8, 24.5])

In [ ]:
begin = PropertyIsLessThanOrEqualTo(propertyname='apiso:TempExtent_begin', literal='2021-04-01 00:00')

In [ ]:
end = PropertyIsLessThanOrEqualTo(propertyname='apiso:TempExtent_end', literal='2021-04-21 23:59')

In [ ]:
cloud = PropertyIsLessThanOrEqualTo(propertyname='apiso:CloudCover', literal='20')

In [ ]:
filter_list = [
    And(
        [
            bbox_query,  # bounding box
            begin, end,  # start and end date
            cloud        # cloud
        ]
    )
]

In [ ]:
csw.getrecords2(constraints=filter_list, outputschema='http://www.isotc211.org/2005/gmd')
csw.results

In [ ]:
collection_query = PropertyIsEqualTo('apiso:ParentIdentifier', 'S2MSI1C')

In [ ]:
csw.getrecords2(constraints=[collection_query], outputschema='http://www.isotc211.org/2005/gmd')
csw.results

In [ ]:
csw.getrecords2(constraints=[bbox_query], outputschema='http://www.isotc211.org/2005/gmd')
csw.results

In [ ]:
for rec in csw.records:
    print(f'identifier: {csw.records[rec].identifier}\ntype: {csw.records[rec].identification.identtype}\ntitle: {csw.records[rec].identification.title}\n')

In [ ]:
anytext_query = PropertyIsEqualTo('csw:AnyText', 'Orthoimagery')

In [ ]:
csw.getrecords2(constraints=[anytext_query])
csw.results

In [ ]:
csw_records = {}
sortby = SortBy([SortProperty('dc:title', 'ASC')])
pagesize=10
maxrecords=1000
startposition = 0
nextrecord = getattr(csw, 'results', 1)
while nextrecord != 0:
    csw.getrecords2(constraints=[anytext_query], startposition=startposition,
                    maxrecords=pagesize, sortby=sortby)
    csw_records.update(csw.records)
    if csw.results['nextrecord'] == 0:
        break
    startposition += pagesize
    if startposition >= maxrecords:
        break
csw.records.update(csw_records)
records = '\n'.join(csw.records.keys())
print('Found {} records.\n'.format(len(csw.records.keys())))
for key, value in list(csw.records.items()):
    print(f'identifier: {value.identifier}\ntype: {value.type}\ntitle: {value.title}\n')

In [ ]:
csw.getrecordbyid(id=['S2B_MSIL2A_20200902T090559_N0214_R050_T34SGH_20200902T113910.SAFE'])

In [ ]:
rec = csw.records['S2B_MSIL2A_20200902T090559_N0214_R050_T34SGH_20200902T113910.SAFE']

In [ ]:
rec.title

In [ ]:
rec.xml

In [ ]:
rec.references

In [ ]:
msg = 'geolink: {geolink}\nscheme: {scheme}\nURL: {url}\n'.format
for ref in rec.references:
    print(msg(geolink=sniff_link(ref['url']), **ref))

In [ ]:
for ref in rec.references:
    url = ref['url']
    if 'WMS' in url:
        print(msg(geolink=sniff_link(url), **ref))
        break

In [ ]:
for ref in rec.references:
    url = ref['url']
    if 'WCS' in url:
        print(msg(geolink=sniff_link(url), **ref))
        break

In [ ]:
print("dataset bbox = (%s, %s, %s, %s)" % (rec.bbox.miny, rec.bbox.minx, rec.bbox.maxy, rec.bbox.maxx))

In [ ]:
m = folium.Map(location=[38, 24], zoom_start=6, tiles='OpenStreetMap')
folium.Rectangle(bounds=[[float(rec.bbox.miny), float(rec.bbox.minx)], [float(rec.bbox.maxy), float(rec.bbox.maxx)]]).add_to(m)
m

### Workspace catalogue

In [ ]:
workspace_endpoint = 'https://resource-catalogue.user.185.52.193.87.nip.io'

In [ ]:
csw = CatalogueServiceWeb(workspace_endpoint,timeout=30)

In [ ]:
csw.getrecords2(maxrecords=20)
csw.results

In [ ]:
for rec in csw.records:
    print(f'identifier: {csw.records[rec].identifier}\ntype: {csw.records[rec].type}\ntitle: {csw.records[rec].title}\n')

In [ ]:
csw.records['INDEX_S2A_MSIL2A_20191216T004701_N0213_R102_T53HPA_20191216T024808'].references

In [ ]:
csw.records['s-expression'].references

In [ ]:
csw.getrecords2(maxrecords=20, distributedsearch=True, hopcount=1)
csw.results

### OpenSearch

In [ ]:
endpoint='http://resource-catalogue.185.52.193.87.nip.io/?service=CSW&version=3.0.0&request=GetCapabilities&mode=opensearch'

In [ ]:
import pyops

In [ ]:
client = pyops.Client(description_xml_url=endpoint)

In [ ]:
results = client.search()
len(results)

In [ ]:
results = client.search(params={"{eo:parentIdentifier?}": {"value": "S2MSI2A"}})
len(results)

In [ ]:
# OpenSearch EO mathematical notation
# n1 equal to field = n1
# {n1,n2,…} equals to field=n1 OR field=n2 OR …
# [n1,n2] equal to n1 <= field <= n2
# [n1,n2[ equals to n1 <= field < n2
# ]n1,n2[ equals to n1 < field < n2
# ]n1,n2] equal to n1 < field  <= n2
# [n1 equals to n1<= field
# ]n1 equals to n1 < field
# n2] equals to field <= n2
# n2[ equals to field < n2

results = client.search(params={"{eo:cloudCover?}": {"value": "80["}})  # eo:cloudCover < 80
len(results)

In [ ]:
results = client.search(params={"{eo:cloudCover?}": {"value": "]20"}})  # eo:cloudCover > 20
len(results)

In [ ]:
results[1]

In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
S = requests.Session()

In [ ]:
R = S.get(url=endpoint)
bs = BeautifulSoup(R.text, 'xml')
print(bs.prettify())

In [ ]:
url='https://resource-catalogue.185.52.193.87.nip.io/?mode=opensearch&service=CSW&version=3.0.0&request=GetRecords&elementsetname=full&resulttype=results&typenames=csw:Record'
R = S.get(url=url)
bs = BeautifulSoup(R.text, 'xml')
print(bs.prettify())

In [ ]:
url='https://resource-catalogue.user.185.52.193.87.nip.io/?mode=opensearch&service=CSW&version=3.0.0&request=GetRecords&elementsetname=full&resulttype=results&typenames=csw:Record'
R = S.get(url=url)
bs = BeautifulSoup(R.text, 'xml')
print(bs.prettify())

## Federated Search

In [ ]:
url='https://resource-catalogue.user.185.52.193.87.nip.io/?mode=opensearch&service=CSW&version=3.0.0&request=GetRecords&typenames=csw:Record&elementsetname=full&resulttype=results&distributedSearch=TRUE&hopcount=1'
R = S.get(url=url)
bs = BeautifulSoup(R.text, 'xml')
print(bs.prettify())

## Collections Search

In [ ]:
url='https://resource-catalogue.185.52.193.87.nip.io/?mode=opensearch&service=CSW&version=3.0.0&request=GetRecords&elementsetname=full&resulttype=results&typenames=csw:Record&eo:parentIdentifier=S2MSI2A'
R = S.get(url=url)
bs = BeautifulSoup(R.text, 'xml')
print(bs.prettify())